In [1]:
# Run if working locally
%load_ext autoreload
%autoreload 2
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
import sqlite3
from sqlite3 import Error
import pickle
import os, sys
import config

config.root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, config.root_path)

from src.dataset.dataset import RawData
from src.dataset.wikisection_preprocessing import (
    tokenize,
    clean_sentence,
    preprocess_text_segmentation,
    format_data_for_db_insertion,
)
from src.dataset.utils import truncate_by_token
from db.dbv2 import Table, AugmentedTable, TrainTestTable
import pprint

from src.bertkeywords.src.similarities import Embedding, Similarities
from src.bertkeywords.src.keywords import Keywords
from src.encoders.coherence_v2 import Coherence
from src.dataset.utils import flatten, dedupe_list, truncate_string

<IPython.core.display.Javascript object>

In [4]:
# initialize the coherence library
coherence = Coherence(max_words_per_step=4)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
No sentence-transformers model found with name /Users/amitmaraj/.cache/torch/sentence_transformers/bert-base-uncased. Creating a new one with MEAN pooling.

<IPython.core.display.Javascript object>

In [5]:
# initialize the keywords and embeddings library
pp = pprint.PrettyPrinter(indent=4)
similarities_lib = Similarities("bert-base-uncased")
keywords_lib = Keywords(similarities_lib.model, similarities_lib.tokenizer)
embedding_lib = Embedding(similarities_lib.model, similarities_lib.tokenizer)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
No sentence-transformers model found with name /Users/amitmaraj/.cache/torch/sentence_transformers/bert-base-uncased. Creating a new one with MEAN pooling.

<IPython.core.display.Javascript object>

In [6]:
dataset_type = "city"
table = Table(dataset_type)
augmented_table = AugmentedTable(dataset_type)
train_test_table = TrainTestTable(dataset_type)

wikisection_city


<IPython.core.display.Javascript object>

In [7]:
data = table.get_all()

text_data = [x[1] for x in data]
text_labels = [x[2] for x in data]

<IPython.core.display.Javascript object>

In [8]:
all_segments = table.get_all_segments()
text_segments = [[y[1] for y in x] for x in all_segments]
segments_labels = [[1 if i == 0 else 0 for i, y in enumerate(x)] for x in all_segments]

<IPython.core.display.Javascript object>

In [9]:
samples = 5
max_tokens = 400

for i, (segment, labels) in enumerate(
    zip(text_segments[:samples], segments_labels[:samples])
):
    for sentence, label in zip(segment, labels):
        # this is the training case. During inference, we will have no idea
        # when segments start and when they end.
        pass

<IPython.core.display.Javascript object>

In [10]:
text_labels[:25]

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

<IPython.core.display.Javascript object>

In [19]:
pruning = 4  # remove the lowest n important words from coherence map
pruning_min = 10  # only prune after n words in the coherence map


def coherence_tester(
    text_data, text_labels, max_tokens=400, max_str_length=30, prediction_thresh=0.3
):
    coherence_map = []
    predictions = []
    for i, (row, label) in enumerate(zip(text_data, text_labels)):
        # compare the current sentence to the previous one
        if i == 0:
            pass
        else:
            prev_row = text_data[i - 1]

            row = truncate_by_token(row, max_tokens)
            prev_row = truncate_by_token(prev_row, max_tokens)

            # add the keywords to the coherence map
            coherence_map.extend(coherence.get_coherence([row, prev_row]))
            print(f"Coherence Map: {[x[0] for x in coherence_map]}")
            if pruning > 0 and len(coherence_map) >= pruning_min:
                print("pruning...", len(coherence_map))
                sorted_map = sorted(
                    coherence_map, key=lambda tup: tup[1]
                )  # sort asc by importance based on keybert
                coherence_map = sorted_map[pruning:][
                    ::-1
                ]  # get the last n - pruning values and reverse the list
                print("done pruning...", len(coherence_map))

            # truncate the strings for printing
            truncated_row = truncate_string(row, max_str_length)
            truncated_prev_row = truncate_string(prev_row, max_str_length)

            # get the keywords for the current sentences
            keywords_current = keywords_lib.get_keywords_with_embeddings(row)
            keywords_prev = keywords_lib.get_keywords_with_embeddings(prev_row)

            # compute the word comparisons between the previous (with the coherence map)
            # and the current (possibly the first sentence in a new segment)
            word_comparisons_with_coherence = embedding_lib.compare_keyword_tuples(
                [*coherence_map, *keywords_prev], keywords_current
            )
            word_comparisons = embedding_lib.compare_keyword_tuples(
                keywords_prev, keywords_current
            )

            similarities_with_coherence = [
                comparison[2] for comparison in word_comparisons_with_coherence
            ]
            avg_similarity_with_coherence = sum(similarities_with_coherence) / len(
                similarities_with_coherence
            )
            similarities = [comparison[2] for comparison in word_comparisons]
            avg_similarity = sum(similarities) / len(similarities)

            # if the two sentences are similar, create a cohesive prediction
            # otherwise, predict a new segment
            if avg_similarity_with_coherence[0] > prediction_thresh:
                print(f"Label: {label}, Prediction: {0}")
                predictions.append((avg_similarity_with_coherence[0], 0))
            else:
                # start of a new segment, empty the map
                coherence_map = []
                print(f"Label: {label}, Prediction: {1}")
                predictions.append((avg_similarity_with_coherence[0], 1))

    #             print(
    #                 f"{[x[0] for x in coherence_map]} <> {[x[0] for x in keywords_prev]} <> {[x[0] for x in keywords_current]}"
    #             )
    #             print(f"{label}, {avg_similarity}, {truncated_prev_row} <> {truncated_row}")
    #             print(
    #                 f"{label}, {avg_similarity_with_coherence}, {truncated_prev_row} <> {truncated_row}"
    #             )

    return predictions

<IPython.core.display.Javascript object>

In [20]:
samples = 6
max_tokens = 400  # want to keep this under 512
max_str_length = 30

coherence_tester(
    text_data[:samples],
    text_labels[:samples],
    max_tokens=max_tokens,
    max_str_length=max_str_length,
)

IndexError: string index out of range

<IPython.core.display.Javascript object>

## Sample Segments

In [15]:
segments_to_test = 10

text_segments_to_check = [
    [truncate_by_token(s, max_tokens) for s in segment]
    for segment in text_segments[:segments_to_test]
]
text_labels_to_check = segments_labels[:segments_to_test]

<IPython.core.display.Javascript object>

In [16]:
len(text_segments_to_check[0]), len(text_labels_to_check[0])

(36, 36)

<IPython.core.display.Javascript object>

In [17]:
different_segment_1 = [
    text_segments_to_check[0][-3],
    text_segments_to_check[0][-2],
    text_segments_to_check[0][-1],
    text_segments_to_check[1][0],
    text_segments_to_check[1][1],
    text_segments_to_check[1][2],
]
different_segment_2 = [
    text_segments_to_check[1][-3],
    text_segments_to_check[1][-2],
    text_segments_to_check[1][-1],
    text_segments_to_check[2][0],
    text_segments_to_check[2][1],
    text_segments_to_check[2][2],
]
different_segment_3 = [
    text_segments_to_check[2][-3],
    text_segments_to_check[2][-2],
    text_segments_to_check[2][-1],
    text_segments_to_check[3][0],
    text_segments_to_check[3][1],
    text_segments_to_check[3][2],
]
different_segment_4 = [
    text_segments_to_check[3][-3],
    text_segments_to_check[3][-2],
    text_segments_to_check[3][-1],
    text_segments_to_check[4][0],
    text_segments_to_check[4][1],
    text_segments_to_check[4][2],
]
different_segment_5 = [
    text_segments_to_check[4][-3],
    text_segments_to_check[4][-2],
    text_segments_to_check[4][-1],
    text_segments_to_check[5][0],
    text_segments_to_check[5][1],
    text_segments_to_check[5][2],
]

same_segment_1 = [
    text_segments_to_check[0][0],
    text_segments_to_check[0][1],
    text_segments_to_check[0][2],
]
same_segment_2 = [
    text_segments_to_check[1][0],
    text_segments_to_check[1][1],
    text_segments_to_check[1][2],
]
same_segment_3 = [
    text_segments_to_check[2][0],
    text_segments_to_check[2][1],
    text_segments_to_check[2][2],
]
same_segment_4 = [
    text_segments_to_check[3][0],
    text_segments_to_check[3][1],
    text_segments_to_check[3][2],
]
same_segment_5 = [
    text_segments_to_check[4][0],
    text_segments_to_check[4][1],
    text_segments_to_check[4][2],
]

<IPython.core.display.Javascript object>

In [18]:
num_sentences_to_check = 6  # test only the first n in the segment
target_labels = [0, 0, 0, 1, 0, 0]

# test coherence on different segments
coherence_tester(
    different_segment_1[:num_sentences_to_check], target_labels[:num_sentences_to_check]
)
coherence_tester(
    different_segment_2[:num_sentences_to_check], target_labels[:num_sentences_to_check]
)
coherence_tester(
    different_segment_3[:num_sentences_to_check], target_labels[:num_sentences_to_check]
)
coherence_tester(
    different_segment_4[:num_sentences_to_check], target_labels[:num_sentences_to_check]
)
coherence_tester(
    different_segment_5[:num_sentences_to_check], target_labels[:num_sentences_to_check]
)

TypeError: get_coherence() got an unexpected keyword argument 'coherence_similarity'

<IPython.core.display.Javascript object>

In [18]:
# test coherence on same segments
coherence_tester(same_segment_1, [0, 0, 0])
coherence_tester(same_segment_2, [0, 0, 0])
coherence_tester(same_segment_3, [0, 0, 0])
coherence_tester(same_segment_4, [0, 0, 0])
coherence_tester(same_segment_5, [0, 0, 0])

Coherence: ['shoreline', 'basque', 'seashore', 'basque']
0, tensor([0.2451]), In spite of appearances both t.. <> The city is in the north of th..
0, tensor([0.3396]), In spite of appearances both t.. <> The city is in the north of th..
Coherence: ['precipitation', 'shoreline', 'climate', 'shoreline']
0, tensor([0.2880]), The city is in the north of th.. <> San Sebastián features an ocea..
0, tensor([0.3303]), The city is in the north of th.. <> San Sebastián features an ocea..
Coherence: ['census', 'sioux', 'area', 'sioux']
0, tensor([0.2409]), Hospers was founded in 1872 wh.. <> Hospers is located at 43 07103..
0, tensor([0.2987]), Hospers was founded in 1872 wh.. <> Hospers is located at 43 07103..
Coherence: ['households', 'census', 'census', 'census']
0, tensor([0.2941]), Hospers is located at 43 07103.. <> As of the census of 2010 there..
0, tensor([0.3422]), Hospers is located at 43 07103.. <> As of the census of 2010 there..
Coherence: ['1943', 'cossacks', '1942', 'cossacks']
0

<IPython.core.display.Javascript object>

In [48]:
# test on a full segment
predictions = coherence_tester(text_segments_to_check[0], text_labels_to_check[0])

Coherence Map: ['shoreline', 'basque', 'seashore', 'basque']
Label: 0, Prediction: 0
Coherence Map: ['shoreline', 'basque', 'seashore', 'basque', 'precipitation', 'shoreline', 'climate', 'shoreline']
Label: 0, Prediction: 0
Coherence Map: ['shoreline', 'basque', 'seashore', 'basque', 'precipitation', 'shoreline', 'climate', 'shoreline', 'paleolithic', 'precipitation', 'evidence', 'precipitation']
pruning... 12
done pruning... 8
Label: 0, Prediction: 1
Coherence Map: ['basque', 'paleolithic', 'roman', 'paleolithic']
Label: 0, Prediction: 1
Coherence Map: ['castile', 'san', 'colonizers', 'san']
Label: 0, Prediction: 0
Coherence Map: ['castile', 'san', 'colonizers', 'san', 'navarre', 'castile', 'iberian', 'castile']
Label: 0, Prediction: 0
Coherence Map: ['castile', 'san', 'colonizers', 'san', 'navarre', 'castile', 'iberian', 'castile', 'neoclassical', 'navarre', 'bourgeois', 'navarre']
pruning... 12
done pruning... 8
Label: 0, Prediction: 1
Coherence Map: ['railway', 'neoclassical', 'bri

<IPython.core.display.Javascript object>